In [3]:
# Jupyter > VSCode

In [4]:
!pip install xlrd
!pip install pynsee
!pip install great_tables

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.8/9.8 MB 17.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 20.7 MB/s eta 0:00:00


In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pynsee
import pynsee.download

In [6]:
np.random.seed(123)

In [7]:
# Importation des données d'émission

url = "https://koumoul.com/s/data-fair/api/v1/datasets/igt-pouvoir-de-rechauffement-global/convert"
emissions = pd.read_csv(url)
emissions.head(2)

,INSEE commune,Commune,Agriculture,Autres transports,Autres transports international,CO2 biomasse hors-total,Déchets,Energie,Industrie hors-énergie,Résidentiel,Routier,Tertiaire
0,01001,L'ABERGEMENT-CLEMENCIAT,3711.425991,NaN,NaN,432.751835,101.430476,2.354558,6.911213,309.358195,793.156501,367.036172
1,01002,L'ABERGEMENT-DE-VAREY,475.330205,NaN,NaN,140.741660,140.675439,2.354558,6.911213,104.866444,348.997893,112.934207


In [8]:
# Sauvegarde des noms des secteurs émetteurs

secteurs = emissions.select_dtypes(include="number").columns

In [10]:
# Importation des données de revenus

from pynsee.download import download_file
filosofi = download_file("FILOSOFI_COM_2016")
filosofi.sample(3)

,CODGEO,LIBGEO,NBMENFISC16,NBPERSMENFISC16,MED16,PIMP16,TP6016,TP60AGE116,TP60AGE216,TP60AGE316,...,PPEN16,PPAT16,PPSOC16,PPFAM16,PPMINI16,PPLOGT16,PIMPOT16,D116,D916,RD16
34640,93077,Villemomble,11968,30123.5,21782.222222222223,61,19,22,23,19,...,23.1,9,5.1,2.1,1.7,1.3,-19.6,9371.481481481482,40950.8,4.36972532901237
9988,28073,Champseru,121,326.5,24806,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
17537,48190,Termes,92,210.5,19108.571428571428,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None


In [13]:
# Conversion en float des variables

filosofi = filosofi.astype({c: "float" for c in filosofi.columns[2:]})
filosofi.sample(3)

,CODGEO,LIBGEO,NBMENFISC16,NBPERSMENFISC16,MED16,PIMP16,TP6016,TP60AGE116,TP60AGE216,TP60AGE316,...,PPEN16,PPAT16,PPSOC16,PPFAM16,PPMINI16,PPLOGT16,PIMPOT16,D116,D916,RD16
17080,47009,Andiran,91.0,226.0,19803.333333,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16528,45102,Conflans-sur-Loing,154.0,360.5,25649.142857,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26679,68046,Bourbach-le-Haut,180.0,434.0,23557.777778,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [39]:
# df.groupby regroupe les individus en fonction d'une variable commune, ici on choisit le département

filosofi["dep"] = filosofi["CODGEO"].str[:2] #on prend les deux premiers numéros de CODGEO
filosofi2 = filosofi.groupby("dep")
# filosofi2.head(2) # Cette commande affiche les deux premiers de CHAQUE groupe !

In [35]:
# Compter le nombre de villes par département

filosofi2 = filosofi.groupby("dep")
filosofi2["LIBGEO"].count() 

dep
01    393
02    800
03    317
04    198
05    162
     ... 
92     36
93     40
94     47
95    184
97     58
Name: LIBGEO, Length: 97, dtype: int64

In [48]:
# Compter la pop. totale française en millions :

In [45]:
filosofi["NBPERSMENFISC16"].sum() * 1e-6
# filosofi["variable"].sum() permet de sommer toutes les valeurs prises par une variable dans toute la table

np.float64(66.9322415)

In [53]:
filosofi.agg({"NBPERSMENFISC16": ["sum", "mean"], "dep" : "count"}).div(1e6)
# filosofi.agg() permet d'obtenir les résultats agrégés souhaités (sum, mean, max, ...)
# Le dictionnaire à l'intérieur permet de l'appliquer directement à plusieurs colonnes

,NBPERSMENFISC16,dep
sum,66.932242,NaN
mean,0.002131,NaN
count,NaN,0.034932


In [54]:
# Combinaison de groupby et agg : compter la pop totale au sein de chaque département

In [59]:
res = filosofi.groupby("dep").agg({"NBPERSMENFISC16" : "sum"})
res

,NBPERSMENFISC16
dep,
01,613088.0
02,514249.0
03,329435.0
04,156537.5
05,133992.5
...,...
92,1583682.0
93,1586664.5
94,1345977.0


In [85]:
# EXO 1
# 1) Calcul des émissions du secteur résidentiel rapporté au département le plus polluant

emissions["dep"] = emissions["INSEE commune"].str[:2]
df1 = emissions.groupby("dep").agg({"Résidentiel" : "sum"}) * 1e-3
df2 = df1.sort_values(by = "Résidentiel", ascending = False)
df2["% de la val. du max"] = df2/(df2.iloc[0])
df2

,Résidentiel,% de la val. du max
dep,,
59,3498.347052,1.000000
75,1934.579997,0.552998
69,1774.653449,0.507283
62,1738.089969,0.496832
57,1644.191719,0.469991
...,...,...
05,122.427395,0.034996
04,116.836087,0.033398
2B,77.109836,0.022042
